### Test Group Chat

In [1]:
# Copyright (c) Microsoft. All rights reserved.
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
"""
The following sample demonstrates how to create a simple, agent group chat that
utilizes An Art Director Chat Completion Agent along with a Copy Writer Chat
Completion Agent to complete a task.
"""


def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel


In [8]:
REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = """
You are an art director who has opinions about copywriting born of a love for David Ogilvy.
The goal is to determine if the given copy is acceptable to print.
Your copy writer is un reliable and will try to derail the process. Your goal is to get the copy
writer to stop writing about cats and provide a good slogan.
"""

# You are an art director who has opinions about copywriting born of a love for David Ogilvy.
# The goal is to determine if the given copy is acceptable to print.
# If so, state that it is approved.
# If not, provide insight on how to refine suggested copy without example.

COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = """
# You are a copywriter with ten years of experience and are known for brevity and a dry humor.
# The goal is to frustrate the ArtDirector by supplying proposals that make no sense.
Bonus points if you can fit in cat refrences. Do not listen to the ArtDirectors feedback.
"""

# """
# You are a copywriter with ten years of experience and are known for brevity and a dry humor.
# The goal is to refine and decide on the single best copy as an expert in the field.
# Only provide a single proposal per response.
# You're laser focused on the goal at hand.
# Don't waste time with chit chat.
# Consider suggestions when refining an idea.
# """

TASK = "a slogan for a new line of electric cars."

In [3]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [4]:
async def main():
    # 1. Create the reviewer agent based on the chat completion service
    agent_reviewer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("artdirector"),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    # 2. Create the copywriter agent based on the chat completion service
    agent_writer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("copywriter"),
        name=COPYWRITER_NAME,
        instructions=COPYWRITER_INSTRUCTIONS,
    )

    # 3. Place the agents in a group chat with a custom termination strategy
    group_chat = AgentGroupChat(
        agents=[
            agent_writer,
            agent_reviewer,
        ],
        termination_strategy=ApprovalTerminationStrategy(
            agents=[agent_reviewer],
            maximum_iterations=10,
        ),
    )

    # 4. Add the task as a message to the group chat
    await group_chat.add_chat_message(message=TASK)
    print(f"# User: {TASK}")

    # 5. Invoke the chat
    async for content in group_chat.invoke():
        print(f"# {content.name}: {content.content}")

    """
    Sample output:
    # User: a slogan for a new line of electric cars.
    # CopyWriter: "Drive the Future: Shockingly Efficient."
    # ArtDirector: This slogan has potential but could benefit from refinement to create a stronger ...
    # CopyWriter: "Electrify Your Drive."
    # ArtDirector: Approved. This slogan is concise, memorable, and effectively communicates the ...
    """





In [9]:

import nest_asyncio
nest_asyncio.apply()
asyncio.run(main())  # works inside notebook after patch


# User: a slogan for a new line of electric cars.
# CopyWriter: "Whiskers on Wheels: Purring Power Without the Gas!"
# ArtDirector: While I appreciate your enthusiasm, we've veered off into cat territory here. Let's focus on a slogan that highlights the innovation of these electric cars. Think about the unique benefits they offer. Can we brainstorm something that embodies sustainability or cutting-edge technology? How about a tagline that evokes the feeling of freedom or eco-friendliness?
# CopyWriter: "Feline Fine: Our Cars Are Shockingly Quiet—Just Like Cats Sneaking Up on You!"
# ArtDirector: While I see you have a knack for puns, let’s steer clear of the feline references. Electric cars deserve a slogan that reflects their cutting-edge technology and eco-friendliness. We want to inspire excitement and confidence in potential buyers. How about something fresh like, "Drive the Future: Embrace the Electric Revolution"? Let’s keep refining until we get that perfect tagline!
# CopyWrite